In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
raw_dataset = pd.read_csv(r"C:\Users\muham\Downloads\365data.csv",quotechar='"',skipinitialspace=True)
dataset = raw_dataset.drop(raw_dataset.columns[[0,1]],inplace=True,axis=1)
dataset = raw_dataset.copy(deep=True)

sixth_prob = [  1,  34,  39,  45,  47,  68,  96, 102, 120, 124, 143, 153, 154, 159,
       174, 195, 208, 217, 247, 249, 259, 260, 262, 275, 279, 281]

dataset[dataset.columns[3]] = dataset[dataset.columns[3]].apply(lambda x: tuple(y.split(" (")[0] if "(" in y else y.strip(",.()") for y in x.split("), ")))
dataset[dataset.columns[9]] = dataset[dataset.columns[9]].apply(lambda x: tuple([y.strip(",. ") for y in str(x).split(".,")]))
dataset[dataset.columns[10]] = dataset[dataset.columns[10]].apply(lambda x: tuple([y.strip(",. ") for y in str(x).split(".,")]))
dataset[dataset.columns[12]] = dataset[dataset.columns[12]].apply(lambda x: tuple(y.strip(". ") for y in x.split(".,")))
dataset[dataset.columns[13]] = dataset[dataset.columns[13]].apply(lambda x: tuple(y.strip(". ") for y in x.split(".,")))
dataset[dataset.columns[6]] = dataset[dataset.columns[6]].apply(lambda x: tuple([y.strip(".,") for y in x.split(".,")]))

dataset[dataset.columns[0]], uniques = pd.factorize(dataset[dataset.columns[0]].explode())
dataset[dataset.columns[1]] = dataset[dataset.columns[1]].map(dict(zip(sorted(dataset[dataset.columns[1]].unique()),list(range(6)))))
dataset[dataset.columns[2]] = dataset[dataset.columns[2]].map(dict(zip(dataset[dataset.columns[2]].unique(),[3,2,4,0,1])))
dataset[dataset.columns[4]] = dataset[dataset.columns[4]].map(dict(zip(sorted(dataset[dataset.columns[4]].unique()),[1,4,5,0,3,2])))
dataset[dataset.columns[5]] = dataset[dataset.columns[5]].map(dict(zip(sorted(dataset[dataset.columns[5]].unique()),[7,6,5,4,3,2,1,0,-1])))
dataset[dataset.columns[7]] = dataset[dataset.columns[7]].map(dict(zip(sorted(dataset[dataset.columns[7]].unique()),[2,3,0,1])))
dataset[dataset.columns[8]] = dataset[dataset.columns[8]].map(dict(zip(sorted(dataset[dataset.columns[8]].unique()),[1,2,3,0,0,0])))
dataset[dataset.columns[11]] = dataset[dataset.columns[11]].map(dict(zip(sorted(dataset[dataset.columns[11]].unique()),[1,0,-1])))

temp = dataset.copy(deep=True)
for i in [3,6,9,10,12,13]:
    temp = temp.explode(temp.columns[i])


weights = temp.groupby(temp.index).count()
weights = 1/weights[weights.columns[0]]
#add weight according to index
temp = temp.merge(weights.rename("Weights"),left_index=True,right_index=True)

fail_answers_three = temp[temp.columns[3]].value_counts().index[10:].to_list()
results = temp[temp.columns[3]].map(lambda x: x in fail_answers_three).reset_index()
#get indeces of true values
indeces = results[results[results.columns[1]] == True].index
#drop rows with indeces
temp.reset_index(inplace=True)
temp = temp.drop(indeces,axis=0)
temp.set_index("index",inplace=True)

temp_factorized = temp.copy(deep=True)
uniques_fac = [i for i in range(6)]
for i,u in zip([3,6,9,10,12,13],range(6)):
    temp_factorized[temp_factorized.columns[i]], uniques_fac[u] = pd.factorize(temp_factorized[temp_factorized.columns[i]])

dataset_wt = dataset.drop(dataset.columns[[3,6,9,10,12,13]],axis=1)

translation = pd.read_csv(r"C:\Users\muham\Downloads\sözlük.txt",header=None).map(lambda x: str(x)).set_index(0).to_dict()[1]
def translator(x):
    return [translation[y] for y in x]
temp_factorized.to_csv("temp.csv",encoding="utf-8",sep="$")